In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
ops.reset_default_graph()
#encoding
#0,1,2
#[S,E,P.......]
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀연습사랑'] 
#                       0,1,2
#범주형 데이터
num_dic={n: i for i, n in enumerate(char_arr)}
dic_len=len(num_dic)
print(dic_len) #41
seq_data=[['word','단어'],['wood','나무'],
          ['game','놀이'],['girl','소녀'],
          ['test','연습'],['love','사랑']]

def make_batch(seq_data):
    input_batch=[]
    output_batch=[]
    target_batch=[]
    for seq in seq_data: #처음 ['word','단어'] 
          input=[num_dic[n] for n in seq[0]] #'word'  =>번호 w: 25, o :20 ,r :17...
          #첫번째 입력되는 데이터는 'word' 를 학습해서 넘어온 smoking gun 
          output=[num_dic[n] for n in ('S' +seq[1])] #훈련된 특징값,'S' 시작점 
          target=[num_dic[n] for n in (seq[1]+'E')] #끝을 확인 하기 위해서 '연습이다'-> '단어EE'
          #identity matrix 단위행렬=>one hot encoding
          #1000000000
          #0100000000
          # ........
          input_batch.append(np.eye(dic_len)[input]) #번호로 만든것 예)25가 들어가면 2차원 배열로 4 x 42 
          output_batch.append(np.eye(dic_len)[output])
          target_batch.append(target)                #2x 41
    return input_batch,output_batch,target_batch
learning_rate=0.01
n_hidden=128   #neurons 출력이 128차
total_epoch=100
n_class=n_input=dic_len #41
#encoder,decoder  용 데이터 주입변수
enc_input=tf.placeholder(tf.float32,[None,None,n_input])
dec_input=tf.placeholder(tf.float32,[None,None,n_input])
targets=tf.placeholder(tf.int64,[None,None])

with tf.variable_scope('encode'): #input4개 
    #셀 자체는 FFNN 한 개 , 입력->가중치->>출력(n_hidden)
    enc_cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden) #입력 41 x 128
    #과적합방지하기 위해 계산회로 생략-random 하게 생략
    enc_cell=tf.nn.rnn_cell.DropoutWrapper(enc_cell,output_keep_prob=0.5,seed=100) 
    #셀이 4개 셀당 41개의 데이터가 입력
    outputs,enc_states=tf.nn.dynamic_rnn(enc_cell,enc_input,dtype=tf.float32)
with tf.variable_scope('decode'): #outputs 3개 
    dec_cell=tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell=tf.nn.rnn_cell.DropoutWrapper(dec_cell,output_keep_prob=0.5,seed=100)
   #2 x 41??
    #앞의 인코더 망에서 출력된 states값:마지막 cell 의 출력값
    #앞의 데이터를 다 고려한 특징 
    #두개의 망을 연결 initial_state=enc_states
    outputs,enc_states=tf.nn.dynamic_rnn(dec_cell,dec_input,initial_state=enc_states,dtype=tf.float32)


41


In [12]:
#전체 데이터가 6개, (output)3개!!가 128개로 나온다 :'S'+'단어'=3개
#(6x3x128 ) x 41 행렬 곱하면 
#6x3x41

model=tf.layers.dense(outputs,n_class,activation=None)
cost=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model,labels=targets))
optimizer=tf.train.AdamOptimizer(learning_rate).minimize(cost)
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
input_batch,output_batch,target_batch=make_batch(seq_data)
for epoch in range(total_epoch): #100회
    _,loss=sess.run([optimizer,cost],feed_dict={enc_input:input_batch,
                                               dec_input:output_batch,
                                               targets:target_batch})
    print('epoch:','%04d' % (epoch + 1),'cost=','{:.6f}'.format(loss)) #epoch + 1 처음이 0부터 시작하기 때문에 

epoch: 0001 cost= 2.693845
epoch: 0002 cost= 1.847687
epoch: 0003 cost= 1.406774
epoch: 0004 cost= 0.630577
epoch: 0005 cost= 0.404428
epoch: 0006 cost= 0.364389
epoch: 0007 cost= 0.223047
epoch: 0008 cost= 0.100556
epoch: 0009 cost= 0.095548
epoch: 0010 cost= 0.082468
epoch: 0011 cost= 0.046243
epoch: 0012 cost= 0.065438
epoch: 0013 cost= 0.020039
epoch: 0014 cost= 0.023188
epoch: 0015 cost= 0.015581
epoch: 0016 cost= 0.016316
epoch: 0017 cost= 0.023700
epoch: 0018 cost= 0.014285
epoch: 0019 cost= 0.006344
epoch: 0020 cost= 0.013925
epoch: 0021 cost= 0.010044
epoch: 0022 cost= 0.004519
epoch: 0023 cost= 0.006793
epoch: 0024 cost= 0.008879
epoch: 0025 cost= 0.000822
epoch: 0026 cost= 0.002353
epoch: 0027 cost= 0.005951
epoch: 0028 cost= 0.002346
epoch: 0029 cost= 0.001693
epoch: 0030 cost= 0.002250
epoch: 0031 cost= 0.002017
epoch: 0032 cost= 0.014444
epoch: 0033 cost= 0.015728
epoch: 0034 cost= 0.001820
epoch: 0035 cost= 0.001198
epoch: 0036 cost= 0.001243
epoch: 0037 cost= 0.001254
e

In [ ]:
#######################

In [16]:
def translate(word): #translate =함수
    #모델의 자리수는 일치시켜야 함 그래서 PPPP
    seq_data=[word,'P' * len(word)] #word, PPPP
    #1x4x41
    #1x4x41 (원핫인코딩)
    input_batch,output_batch,target_batch=make_batch([seq_data])
    
    #1x5x41 : (PPPP+E =5)
    prediction=tf.argmax(model,2) #면, 행, 렬 중 열을 중심으로 (41개중 max)
    result=sess.run(prediction,feed_dict={enc_input: input_batch,
                                         dec_input:output_batch,
                                         targets:target_batch})
    decoded=[char_arr[i] for i in result[0]] #5 x 41
    end=decoded.index('E')
    translated=''.join(decoded[:end])
    
    return translated
print('word->',translate('word'))

word-> 단어


In [ ]:
#OCR: kakao API
#카카오 

In [21]:
import json
import cv2 #computer vision 이미지, 동영상처리
import requests #html 페이지를 요청
import sys
LIMIT_PX=1024
LIMIT_BYTE=1024*1024
LIMIT_BOX=40

#ocr 절차
#이미지사이즈 조정->글씨를 둘러싼 box를 detection (컬러를 찾아서 박스로 잘라냄)

In [23]:

def kaka0_ocr_resize(image_path:str): #규격이미지 사이즈
    image=cv2.imread('2.jpg') #자기의 이미지로 변경:영수증
    height,width,_=image.shape
    if LIMIT_PX < height or LIMIT_PX < width:
        ratio=float(LIMIT_PX) / max(height,width)
        image=cv2.resize(image,None,fx=ratio,fy=ratio)
        height,width,_=height,width,_=image.shape
        image_path='{}_resized.jpg'.format('2')
        cv2.imwrite(image_path,image)
        return image_path
    return None

In [ ]:
#카카오에서 준 아이디를 기입
인코딩할때 jpg 로 

In [24]:
#사용법
#open api 함수를 원격호출(remote call)->네트워크를 통해서 requests 호출(시간이 걸림: 비동기방식
#비동기방식:호출시키고 돌아올때까지 다른 작업을 하고 있음- event를 받아서 
#대표적인 비동기 통신방식: AJAX :웹브라우저 내부적 처리

def kakao_ocr_detect(image_path:str,appkey:str):
    API_URL='https://kapi.kakao.com/v1/vision/text/detect'
    headers={'authorization':'KakaoAK{}'.format('자신의 API_ID')}
    #naver=>jpg 혁식만, gif(애니메이션), png(transparent 지원)
    image=cv2.imread('2.jpg') #cv2 통해서 이미지를 로드할때
    jpeg_image=cv2.imecode('.jpg',image)[1]
    data=jpeg_image.tobytes() #네크워크 전송->serialization
    
    #네트워크일때->데이터가시리얼로 가야함(네크워크모델)
    #8개 line 으로 구성=>4개 회선만 사용->1개 전송 ,1개는 수신
    #flask에서 사용
    #인서텟에서 표준 데이터 포멧:json 
    return requests.post(API_URL,headers=headers,files={'file':data},data={'boxes':json.dumps(boxes)})

In [25]:
#자동 가계부
def main():
    image_path,appkey=sys.argv[0],sys.argv[0]
    resize_impath=kakao_ocr_resize(image_path)
    output=kakao_ocr_detect(image_path,appkey).json()
    boxes=output['result']['boxes']
    boxes=boxes[:min(len(boxes),LIMIT_BOX)]
    output=kakao_ocr_recognize(image_path,boxes,appkey).json()
output=main()
output['result']

NameError: name 'kakao_ocr_resize' is not defined

In [ ]:
import json

import cv2
import requests
import sys

LIMIT_PX = 1024
LIMIT_BYTE = 1024*1024  # 1MB
LIMIT_BOX = 40


def kakao_ocr_resize(image_path: str):
    """
    ocr detect/recognize api helper
    ocr api의 제약사항이 넘어서는 이미지는 요청 이전에 전처리가 필요.

    pixel 제약사항 초과: resize
    용량 제약사항 초과  : 다른 포맷으로 압축, 이미지 분할 등의 처리 필요. (예제에서 제공하지 않음)

    :param image_path: 이미지파일 경로
    :return:
    """
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    if LIMIT_PX < height or LIMIT_PX < width:
        ratio = float(LIMIT_PX) / max(height, width)
        image = cv2.resize(image, None, fx=ratio, fy=ratio)
        height, width, _ = height, width, _ = image.shape

        # api 사용전에 이미지가 resize된 경우, recognize시 resize된 결과를 사용해야함.
        image_path = "{}_resized.jpg".format(image_path)
        cv2.imwrite(image_path, image)

        return image_path
    return None


def kakao_ocr_detect(image_path: str, appkey: str):
    """
    detect api request example
    :param image_path: 이미지파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/detect'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data})


def kakao_ocr_recognize(image_path: str, boxes: list, appkey: str):
    """
    recognize api request example
    :param boxes: 감지된 영역 리스트. Canvas 좌표계: 좌상단이 (0,0) / 우상단이 (limit,0)
                    감지된 영역중 좌상단 점을 기준으로 시계방향 순서, 좌상->우상->우하->좌하
                    ex) [[[0,0],[1,0],[1,1],[0,1]], [[1,1],[2,1],[2,2],[1,2]], ...]
    :param image_path: 이미지 파일 경로
    :param appkey: 카카오 앱 REST API 키
    """
    API_URL = 'https://kapi.kakao.com/v1/vision/text/recognize'

    headers = {'Authorization': 'KakaoAK {}'.format(appkey)}

    image = cv2.imread(image_path)
    jpeg_image = cv2.imencode(".jpg", image)[1]
    data = jpeg_image.tobytes()

    return requests.post(API_URL, headers=headers, files={"file": data}, data={"boxes": json.dumps(boxes)})


def main():
    if len(sys.argv) != 3:
        print("Please run with args: $ python example.py /path/to/image appkey")
    #image_path, appkey = sys.argv[1], sys.argv[2]
    image_path, appkey = "2.jpg","0166edbccc8fae3043dc6f0830749d81"

    resize_impath = kakao_ocr_resize(image_path)
    if resize_impath is not None:
        image_path = resize_impath
        print("원본 대신 리사이즈된 이미지를 사용합니다.")

    output = kakao_ocr_detect(image_path, appkey).json()
    print("[detect] output:\n{}\n".format(output))

    boxes = output["result"]["boxes"]
    boxes = boxes[:min(len(boxes), LIMIT_BOX)]
    output = kakao_ocr_recognize(image_path, boxes, appkey).json()
    print("[recognize] output:\n{}\n".format(json.dumps(output, sort_keys=True, indent=2)))


if __name__ == "__main__":
    main()


In [ ]:
#카카오 개발자 
https://developers.kakao.com/apps/393146/created